# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [28]:
# Импортируем необходимые библиотеки и модули
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [29]:
# Считаем данные из датасета, проверим корректность выгрузки
scoring_data = pd.read_csv('/datasets/data.csv')
scoring_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [30]:
# Расмотрим общую информацию о данных
scoring_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов. Они содержат следующую информацию:
* *children* — количество детей в семье
* *days_employed* — общий трудовой стаж в днях
* *dob_years* — возраст клиента в годах
* *education* — уровень образования клиента
* *education_id* — идентификатор уровня образования
* *family_status* — семейное положение
* *family_status_id* — идентификатор семейного положения
* *gender* — пол клиента
* *income_type* — тип занятости
* *debt* — имел ли задолженность по возврату кредитов
* *total_income* — ежемесячный доход
* *purpose* — цель получения кредита  

Количество значений в столбцах различается. Это говорит о том, что в данных есть нулевые значения.

In [31]:
# Расмотрим сводную информацию о данных
scoring_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Сразу виден ряд проблем с данными:  
* столбец *education* содержит дубликаты, написанные в разных регистрах
* столбец *total_income* содержит излишне подробную информацию, вплоть до 6 знака после запятой
* столбец *children* содержит отрицательные значения (-1) и слишком высокие (20)
* столбец *dob_years* содержит нулевые значения  
* столбец *days_employed*  содержит большое количество отрицательных значений, а также слишком большие положительные

Рассмотрим последний столбец подробнее.

In [32]:
# Узнаем информацию по трудовому стажу
negative_days_employed = scoring_data[scoring_data['days_employed']<0]['days_employed'].count()

print ('Количество строк с отрицательным трудовым стажем: ', negative_days_employed )
print ('Доля строк с отрицательным трудовым стажем: {:.2%}' .format( negative_days_employed / scoring_data.shape[0] ))

Количество строк с отрицательным трудовым стажем:  15906
Доля строк с отрицательным трудовым стажем: 73.90%


### Вывод

Из предварительного анализа можно сделать следующие выводы:
* Каждая строка содержит информацию об одном клиенте банка
* В данных встречаются пропуски и дубликаты, которые необходимо обработать
* В данных встречаются выбросы и артефакты, их также необходимо обработать
* Столбец *days_employed* содержит почти 74% отрицательных, то есть некорректных, значений. Поэтому нецелесобразно использовать его в дальнейшем анализе.

Скорее всего, некорректные данные появились, так как произошла ошибка при выгрузке данных на стороне банка.

## Шаг 2. Предобработка данных

### Обработка пропусков

Узнаем сколько пропусков содержится в данных, какую долю от общего количества данных они составляют,  и есть ли между ними связь

In [33]:
scoring_data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [34]:
scoring_data[ scoring_data['total_income'].isnull() & scoring_data['days_employed'].isnull() ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [35]:
print ('Доля пропусков: {:.2%}' .format( scoring_data['days_employed'].isnull().sum()  / scoring_data.shape[0] ))

Доля пропусков: 10.10%


**Промежуточный вывод:**  
* Всего пропусков 2174.
* Пропуски в столбцах *days_employed* и *total_income* взаимосвязаны и встречаются только одновременно
* Доля пропусков составляет около 10% от общего числа данных. Это достаточно большая доля, поэтому пропуски нельзя удалить, и их необходимо обработать - подставить корректные значения.

Пропуски в столбце *days_employed* не играют роли, так как мы в дальнейшем не будем использовать этот столбец. Однако для избежания дальнейших ошибок при обработке, заменим пропуски на нули.
Пропуски в столбце *total_income* можно заменить либо на среднее арифметическое значение, либо на медиану. Чтобы понять, что лучше подходит в данной ситуации, посмотрим на разброс зарплат.

In [36]:
print ('Минимальная зарплата', scoring_data['total_income'].min())
print ('Максимальная зарплата', scoring_data['total_income'].max())

Минимальная зарплата 20667.26379327158
Максимальная зарплата 2265604.028722744


Разброс очень большой, поэтмоу целесообразнее применить медиану.

In [37]:
# Посчитаем и выведем медиану
total_income_median = scoring_data['total_income'].median()
total_income_median

145017.93753253992

In [38]:
# Заполним пропуски
scoring_data['total_income'] = scoring_data['total_income'].fillna(value=total_income_median)
scoring_data['days_employed'] = scoring_data['days_employed'].fillna(value=0)

In [39]:
# Проверим данные
scoring_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Обработка некорректных значений  
Помимо пропусков, в датасете встречаются и другие некорректные данные. Изучим и обработаем их.

In [40]:
# Изучим столбец gender на наличие ошибок
scoring_data['gender'].value_counts() 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Видим одну строку с ошибочным значением *XNA*. Мы не знаем настоящий пол этого клиента, поэтому заменим значение на пустую строку.  
При анализе влияния пола на целевые показатели, мы не будем учитывать эту строку.

In [41]:
# Подставим пустую строку в ячейку, и проверим результат
scoring_data.loc[scoring_data['gender']=='XNA', 'gender'] = ''
scoring_data['gender'].value_counts() 

F    14236
M     7288
         1
Name: gender, dtype: int64

In [42]:
# Узнаем сколько строк, где возраст клиента равен нулю
scoring_data[scoring_data['dob_years'] == 0]['dob_years'].count()

101

In [43]:
# Заменим нули на средний возраст клиентов и проверим результат.
scoring_data['dob_years'].replace(0, round(scoring_data['dob_years'].mean()), inplace=True)
scoring_data['dob_years'].min()

19

In [44]:
# Узнаем информацию по количеству детей
scoring_data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [45]:
# Заменим некорректные значения на среднее количество клиентов, округленное до целой части, и проверим результат.
scoring_data['children'].replace(-1, round(scoring_data['children'].mean()), inplace=True)
scoring_data['children'].replace(20, round(scoring_data['children'].mean()), inplace=True)
scoring_data['children'].value_counts()

0    14149
1     4941
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

* Мы обработали пропуски, заменив пустые значения на 0 в столбце *days_employed*, и на медианное значение в столбце *total_income*.
* Мы заменили некорректные значения в столбцах *children* и *dob_years* на средние значения.


### Замена типа данных

Обработаем столбец *total_income*.  
Во-первых, додход измеряется в рублях, и поэтому максимальная точность может быть только до второго знака после запятой.  
Во-вторых, учитывая масштаб и размеры цифр, копейки не играеют существенной роли, но при этом затрудняют восприятие и анализ информации.
Поэтому целесообразно округлить значения и перевести их в целочисленный тип. Для этого применим метод `.astype('int')`.

In [46]:
# Посмотрим, как выглядит столбец до обработки
scoring_data['total_income'].head()

0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
4    158616.077870
Name: total_income, dtype: float64

In [47]:
# Переведем значения в целочисленный тип и проверим результат.
scoring_data['total_income'] = scoring_data['total_income'].astype('int')
scoring_data['total_income'].head()

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

### Вывод

Мы перевели значения столбца *total_income* из float64 в int64.

### Обработка дубликатов

Мы уже знаем что столбец *education* содержит дубликаты, котрые отличаются регистром. Переведем все слова в нижний регистр.

In [48]:
# Посмотрим, как выглядит столбец до обработки
scoring_data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [49]:
# Переведем значения в нижний регистр и проверим результат.
scoring_data ['education'] = scoring_data['education'].str.lower()
scoring_data ['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим весь датасет на наличие дубликатов, то есть повторяющихся строк.

In [50]:
# Узнаем сколько дубликатов всего 
scoring_data.duplicated().sum()

71

In [51]:
# Удалим дубликаты, сбросим нумерацию индексов и проверим результат
scoring_data = scoring_data.drop_duplicates().reset_index(drop = True)
scoring_data.duplicated().sum()

0

### Вывод

* Мы избавились от дубликатов в столбце *education*, приеведя все к единому регистру.
* Мы избавились от повторяющихся строк 

### Лемматизация

In [52]:
# Изучим столбец purpose 
scoring_data['purpose'].value_counts()   

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В данном столбце слишком много вариантов цели кредита. В таком виде данные трудно анализировать, поэтому целесообращно применить лемматизацию, чтобы привести спсиок к более короткому и читаемому виду.

Осмотр перечня причин позволяет выделить ряд наиболее частых причин, а именно: свадьба, образование, жилье/недвижимость и автомобиль.
Жилье и недвижимость - это два синонима. В словаре оставим только слово 'недвижимость'. При лемматизации, в случае нахождения слова 'жилье', будем подставлять в результат слово 'недвижимость'.

In [53]:
# Создадим список из наиболее частых уникальных причин для кредита
unique_purposes = ['свадьба', 'образование', 'недвижимость', 'автомобиль' ] 

In [54]:
# Создадим объект лемматизатор
m = Mystem()

# Создадим функцию для сопоставления причины со списком уникальных причин
def find_clear_purpose(purpose):
      for clear_purpose in unique_purposes:
        if clear_purpose in m.lemmatize(purpose):
            return clear_purpose
        if 'жилье' in m.lemmatize(purpose):
            return 'недвижимость'

In [55]:
# Создадим отдельный столбец с результами работы функции.
scoring_data['cleared_purpose'] = scoring_data['purpose'].apply(find_clear_purpose)

In [56]:
# Проверим результаты обработки
scoring_data[['purpose', 'cleared_purpose']].head(10)

,purpose,cleared_purpose
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
5,покупка жилья,недвижимость
6,операции с жильем,недвижимость
7,образование,образование
8,на проведение свадьбы,свадьба
9,покупка жилья для семьи,недвижимость


In [57]:
# Проверим результаты обработки
scoring_data['cleared_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: cleared_purpose, dtype: int64

### Вывод

Применив лемматизацию, мы сократили количество причин до четырех. Можно заметить, что иппотечные кредиты ('недвижимость') являются наиболее распространенными.

### Категоризация данных

Перед нами поставлены четыре целевых вопроса:  
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?  

Проведем категоризацию и группировку данных в соответсвии с данными вопросами.  

При дальнейшей работе будем оттакиваться от следующих рабочих гипотез:
* Наличие детей повышает вероятность возврата кредита.
* Полноценные семьи чаще возвращают кредиты. 
* Люди с низким уровнем дохода с большей вероятностью становятся должниками
* По иппотечным кредитам (на недвижимость) и образованию меньше всего задолженностей.

Сгруппируем данные в три таблицы: по количеству детей, по семейному статусу, по цели кредита и по уровню дохода.  
В качестве целевой переменной будем рассматривать долю задолженностей в процентах в каждой группе - это будет вероятность невозврата. 
На абсолютные числа должников в группах нельзя ориентироваться, так как они могут зависеть от того, что данные социально-демографические группы больше сами по себе или генерируют больше заявок на кредиты.

#### Категоризация по уровню дохода  
Дя более простого и наглядного анализа по уровню дохода разобъем все данные на три категории в зависимости от суммы заработка: низший класс, средний класс и высший класс.  

Согласно [данным источника](https://www.rbc.ru/economics/18/06/2018/5b27bf829a794710796bbfe5) минимальный уровень зарплат для причисления к среднему классу в Москве составляет 120 тыс. руб. (до вычета налогов), в остальных субъектах РФ, для того чтобы считаться средним классом, россиянам необходимо зарабатывать в среднем как минимум 60 тыс. руб. Высший класс начинается при заработке от 200 тыс. руб. в Москве, и от 100 тыс. руб. в регионах.  

Чтобы понять Московский у нас банк или региональный, изучим краткую сводку по столбцу *total_income*.

In [58]:
# Приведем вывод к целочисленному типу
scoring_data.describe()['total_income'].astype('int')

count      21454
mean      165225
std        98021
min        20667
25%       107623
50%       145017
75%       195813
max      2265604
Name: total_income, dtype: int64

Видим большая часть зарплат лежит в диапазоне свыше 100 тыс. руб.  
Поэтому будем придерживаться предположения, что речь идет о Московско банке и, соответсвтенно, Московских порогах социальных классов.

In [59]:
# Установим планки социальных классов в 100 и 200 тыс. руб.
middle_income = 100000
high_income = 200000

In [60]:
# Создадим функцию для определения принадлежности к классу
def social_class_definer (total_income):
    if total_income <= middle_income:
        return 'низший класс'
    if total_income <= high_income:
        return 'средний класс'
    return 'высший класс'

In [61]:
# Применим функцию к дадасету, результат занесем в новый столбец и проверим корректность работы.
scoring_data['social_class'] = scoring_data['total_income'].apply(social_class_definer)
scoring_data['social_class'].value_counts()

средний класс    11925
высший класс      5066
низший класс      4463
Name: social_class, dtype: int64

In [62]:
# Сгруппируем данные и проверим их.
#
# 'count' - общее колтичество семей в группе
# 'sum' - колтичество должников в группе
# 'mean' - доля должников в группе
social_class_info = scoring_data.groupby('social_class').agg({'debt':['count','sum', 'mean']})
social_class_info

debt                
               count   sum      mean
social_class                        
высший класс    5066   358  0.070667
низший класс    4463   354  0.079319
средний класс  11925  1029  0.086289

#### Группировка данных по детям  
Сгруппируем информацию по количеству детей в семье. Посчитаем, сколько всего таких семей, сколько среди них должников и какой процент они составляют.

In [63]:
# Сгруппируем данные и проверим их.
#
# 'count' - общее колтичество семей в группе
# 'sum' - колтичество должников в группе
# 'mean' - доля должников в группе

children_info = scoring_data.groupby('children').agg({'debt':['count', 'sum']})
children_info

debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4931   453  0.091868
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

#### Группировка данных по цели кредита  
Сгруппируем информацию по цели кредита. Посчитаем, сколько всего таких клиентов в этих группах, сколько среди них должников и какой процент они составляют.

In [64]:
# Сгруппируем данные и проверим их.
#
# 'count' - общее колтичество семей в группе
# 'sum' - колтичество должников в группе
# 'mean' - доля должников в группе

purpose_info = scoring_data.groupby('cleared_purpose').agg({'debt':['count', 'sum', 'mean']})
purpose_info

debt               
                 count  sum      mean
cleared_purpose                      
автомобиль        4306  403  0.093590
недвижимость     10811  782  0.072334
образование       4013  370  0.092200
свадьба           2324  186  0.080034

#### Группировка данных по семейному статусу  

Сгруппируем информацию по семейному статусу. Посчитаем, сколько всего таких семей, сколько среди них должников и какой процент они составляют.

In [65]:
# Сгруппируем данные и проверим их.
#
# 'count' - общее колтичество семей в группе
# 'sum' - колтичество должников в группе
# 'mean' - доля должников в группе

family_info = scoring_data.groupby('family_status').agg({'debt':['count', 'sum', 'mean']})
family_info

debt               
                       count  sum      mean
family_status                              
Не женат / не замужем   2810  274  0.097509
в разводе               1195   85  0.071130
вдовец / вдова           959   63  0.065693
гражданский брак        4151  388  0.093471
женат / замужем        12339  931  0.075452

#### Подготовка сводных таблиц  
Подготовим сводные таблицы для финального анализа.

In [66]:
# Введем вспомогательный столбец, в который внесем информацию о наличие детей как таковых.
# False -  если детей нет 
# True - если дети есть

scoring_data['have_children'] = scoring_data['children'] > 0

In [67]:
# Создадим таблицу, демонстрирующую зависимость семейного статуса и наличия детей

family_and_children = pd.pivot_table(scoring_data, index='family_status',
                            columns='have_children', values='debt', aggfunc='mean')
family_and_children.sort_values(by=False, ascending=False)

have_children,False,True
family_status,,
Не женат / не замужем,0.092838,0.116788
гражданский брак,0.083883,0.111893
в разводе,0.070153,0.072993
женат / замужем,0.069095,0.085198
вдовец / вдова,0.062574,0.089286


In [68]:
# Добавим к таблице столбец, содержащий разницу между столбцами с детьми и без них
family_and_children['difference'] = family_and_children[True] - family_and_children[False]

In [69]:
# Создадим таблицу, демонстрирующую зависимость социального положения и наличия детей

social_class_and_children = pd.pivot_table(scoring_data, index='social_class',
                            columns='have_children', values='debt', aggfunc='mean')
social_class_and_children.sort_values(by=False, ascending=False)

have_children,False,True
social_class,,
средний класс,0.079789,0.098503
низший класс,0.072435,0.094193
высший класс,0.067854,0.075732


In [70]:
# Добавим к таблице столбец, содержащий разницу между столбцами с детьми и без них
social_class_and_children['difference'] = social_class_and_children[True] - social_class_and_children[False]

### Вывод

Мы категоризировали данные по уровню дохода, сгруппировали данные по количеству детей и семейному статусу, подготовили сводные таблицы для заключительного анализа.

## Шаг 3. Ответьте на вопросы

### Наличие детей и семейное положение

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [71]:
# Взглянем еще раз на данные по количеству детей в семьях
children_info

debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4931   453  0.091868
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

In [72]:
# Выведем процентное соотношение должников по каждой категории
print (round (children_info['debt', 'mean'].sort_values(),3)*100 )

children
5    0.0
0    7.5
3    8.2
1    9.2
2    9.5
4    9.8
Name: (debt, mean), dtype: float64


In [73]:
family_and_children

have_children,False,True,difference
family_status,,,
Не женат / не замужем,0.092838,0.116788,0.023950
в разводе,0.070153,0.072993,0.002840
вдовец / вдова,0.062574,0.089286,0.026712
гражданский брак,0.083883,0.111893,0.028010
женат / замужем,0.069095,0.085198,0.016103


In [74]:
print (round (family_and_children['difference'].sort_values(),3)*100 )

family_status
в разводе                0.3
женат / замужем          1.6
Не женат / не замужем    2.4
вдовец / вдова           2.7
гражданский брак         2.8
Name: difference, dtype: float64


In [75]:
# Выведем процентное соотношение должников по каждой категории
social_class_and_children

have_children,False,True,difference
social_class,,,
высший класс,0.067854,0.075732,0.007879
низший класс,0.072435,0.094193,0.021757
средний класс,0.079789,0.098503,0.018714


In [76]:
print (round (social_class_and_children['difference'].sort_values(),3)*100 )

social_class
высший класс     0.8
средний класс    1.9
низший класс     2.2
Name: difference, dtype: float64


### Вывод

* Столбец **difference** из таблиц *family_and_children* и *social_class_and_children* в обоих случаях содержит только положительные значения, то есть прирост значения. Таким образом, получаем, что наличие детей повышает вероятность задолженности по кредиту. 
* Бездетные семьи и семьи с тремя детьми чаще всего возвращают кредиты. У семей с двумя и четыремя детьми повышенная вероятность возникновения задолженности.
* Семьи с пятью детьми имеют нулевую вероятность задолженности, которая появляется из-за того, что среди данных нет ни одного зафиксированного случая задолженности. Однако абсолютный размер данной выборки слишком мал, поэтому относительно нее нельзя делать никаких выводов.

### Семейное положение

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [77]:
# Взглянем еще раз на данные по уровню дохода и социальному статусу
family_info

debt               
                       count  sum      mean
family_status                              
Не женат / не замужем   2810  274  0.097509
в разводе               1195   85  0.071130
вдовец / вдова           959   63  0.065693
гражданский брак        4151  388  0.093471
женат / замужем        12339  931  0.075452

In [78]:
# Выведем процентное соотношение должников по каждой категории
round (family_info['debt', 'mean'].sort_values(),3)*100

family_status
вдовец / вдова           6.6
в разводе                7.1
женат / замужем          7.5
гражданский брак         9.3
Не женат / не замужем    9.8
Name: (debt, mean), dtype: float64

### Вывод

Чаще всего кредит не возвращают не женатые/не замужние, а также семьи, состоящие в гражданском браке. С большей вероятностью кредит вылачивают вдовы/вдовцы и разведенные.

###  Уровень дохода

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [79]:
# Взглянем еще раз на данные по уровню дохода и социальному статусу
social_class_info

debt                
               count   sum      mean
social_class                        
высший класс    5066   358  0.070667
низший класс    4463   354  0.079319
средний класс  11925  1029  0.086289

In [80]:
# Выведем процентное соотношение должников по каждой категории
round (social_class_info['debt', 'mean'].sort_values(),3)*100

social_class
высший класс     7.1
низший класс     7.9
средний класс    8.6
Name: (debt, mean), dtype: float64

### Вывод

Представители среднего класса чаще всего не возвращают кредиты.  
Представители высшего класс реже всех становятся должниками. Разница между высшим и низшим классом составляет 0.8%

### Цель кредита

- Как разные цели кредита влияют на его возврат в срок? 

In [81]:
# Взглянем еще раз на данные по целям кредита
purpose_info

debt               
                 count  sum      mean
cleared_purpose                      
автомобиль        4306  403  0.093590
недвижимость     10811  782  0.072334
образование       4013  370  0.092200
свадьба           2324  186  0.080034

In [82]:
# Выведем процентное соотношение должников по каждой категории
round (purpose_info['debt', 'mean'].sort_values(),3)*100

cleared_purpose
недвижимость    7.2
свадьба         8.0
образование     9.2
автомобиль      9.4
Name: (debt, mean), dtype: float64

### Вывод  
Кредиты на недвижимость и на свадьбу возвращают чаще чем кредиты на образование и на автомобиль.

## Шаг 4. Общий вывод

Перед анализом целевых вопросов были выдвинуты следующие рабочие гипотезы:
1. Наличие детей повышает вероятность возврата кредита.
1. Полноценные семьи чаще всего возвращают кредиты. 
1. Люди с низким уровнем дохода с большей вероятностью становятся должниками
1. По иппотечным кредитам (на недвижимость) и образованию меньше всего задолженностей.  

**1 Гипотеза** не подтвердилась: наличие детей повышает вероятность задолженности в среднем на 2%.  
**2 Гипотеза** не подтвердилась: чаще всего кредиты возвращают люди, пережившие потерю супруга, и разведенные.  
**3 Гипотеза** частично подтвердилась: представителли низшего класса находятся на втором месте по задолженностям. На первом месте, с опережением на 1.3%, находятся представители среднего класса.  
**4 Гипотеза** частично подтвердилась: по иппотеке действительно самый малый процент должников, однако вероятность возникновения должников по образованию на 2% больше, и образование находится на втором месте из четырех.  

В дальнейшем обработанный датасет и полученные результаты можно использовать для обучения классификатора, при построении модели кредитного скоринга.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.  